In [1]:
import pandas as pd
import os
import glob
from datetime import datetime
import openpyxl 
from openpyxl.styles import PatternFill
from openpyxl.styles import Font
from openpyxl.styles import Alignment
from openpyxl.styles import Border, Side

double = Side(border_style="double", color="4617F1")
thin = Side(border_style="thin", color="4617F1")
regular = Side(border_style="medium", color="000000")
time = datetime.now()

from tkinter import *
from tkinter import filedialog
import os
import sys


def browseFiles():
    root = Tk()
    root.withdraw()
    folder_selected = filedialog.askdirectory()
    return folder_selected
window = Tk()
button_explore = Button(window,text = "Browse Files", command = browseFiles)
window.withdraw()   


folderPath = browseFiles()

def color():
    wb = openpyxl.load_workbook(filePath) #     ws = wb['ACT16105']
    clinstoreStudiesPath = folderPath +'/Clstr/'
    clinstorecsv_files = glob.glob(os.path.join(clinstoreStudiesPath, "*.txt"))
    for f in clinstorecsv_files:
            sheetname = f.split("/")[-1].split("_")[-2]
            ws = wb[sheetname]
            for c in ws['A2:H2'][0]:
                c.border = Border(top=regular,bottom=regular,left=regular,right=regular)
            fill_pattern = PatternFill(patternType='solid', fgColor='2f72b1')
            for row in ws.iter_rows(min_row=1, max_row=1, min_col=1, max_col=8):
              for cell in row:
                cell.fill = fill_pattern # ws['B4'].fill = fill_pattern
                ws.row_dimensions[1].height = 30
                ws.column_dimensions['A'].width = 25
                ws.column_dimensions['B'].width = 25
                ws.column_dimensions['C'].width = 30
                ws.column_dimensions['D'].width = 1
                ws.column_dimensions['E'].width = 20
                ws.column_dimensions['F'].width = 20
                ws.column_dimensions['G'].width = 20
                ws.column_dimensions['H'].width = 20
                ws.merge_cells('A1:A2')
                ws.merge_cells('B1:C1')
                ws.merge_cells('E1:G1')
                ws.merge_cells('H1:H2')
                center_align = Alignment(horizontal='center', vertical='center')
                ws.cell(row = 1, column = 2).font = Font(size = 11, bold = True, color = 'F2F2F2')
                ws.cell(row = 1, column = 5).font = Font(size = 11, bold = True, color = 'F2F2F2')
                ws.cell(row = 1, column = 1).font = Font(size = 11, bold = True, color = 'F2F2F2')
                ws.cell(row = 1, column = 8).font = Font(size = 11, bold = True, color = 'F2F2F2')
                ws.cell(row = 1, column = 2).alignment = center_align
                ws.cell(row = 1, column = 5).alignment = center_align
                ws.cell(row = 1, column = 1).alignment = center_align
                ws.cell(row = 1, column = 8).alignment = center_align
                ws.cell(row = 1, column = 1).value = 'Domain Name'
                ws.cell(row = 1, column = 2).value = 'Clinstore'
                ws.cell(row = 1, column = 5).value = 'PDR'
                ws.cell(row = 1, column = 8).value = 'Count Difference'
#                 ws['H3'] = '= IMSUB(C3,G3)'
                for i in range(3, ws.max_row+1):
                   # ws['H'+str(i)] = '= IMSUB(C"+str(i)",G"+str(i)")' 
                   ws.cell(row = i, column = 8).value = f'= IMSUB(C{i} ,G{i})'


    wb.save(filePath)

def clinstorecsv():
    clinstoreStudiesPath = folderPath +'/Clstr/'
    clinstorelist = []
    clinstorecsv_files = glob.glob(os.path.join(clinstoreStudiesPath, "*.txt"))
    
    for f in clinstorecsv_files:
        data = pd.read_csv(f)    
        data.rename(columns = {data.columns[0]:'TEST'}, inplace = True)
        A = data['TEST'].str.split()
        df = pd.DataFrame([A][0])
        B = [item for item in A]
        df = pd.DataFrame(B, columns = ['Clinstore Domain Name', 'Clinstore Record Count','Clinstore Active Record Count'])      
        df = df.astype({"Clinstore Record Count": int})
        if '?' in df['Clinstore Active Record Count'].values :
            df['Clinstore Active Record Count'] = df['Clinstore Active Record Count'].replace('?','0')
        df = df.astype({"Clinstore Active Record Count": int})
        clinstorelist.append(df)

    with pd.ExcelWriter(filePath) as writer:
        count=0
        for f in clinstorecsv_files:
            sheetname = f.split("/")[-1].split("_")[-2]
            clinstorelist[count].to_excel(writer, sheet_name=sheetname, header= True,startrow=1, index=False)            
            count = count+1 
            print("\tData Copied for Clinstore Studies: ", sheetname)
    print("\nSuccess...")
    print("Final Excel Sheet generated location : ",filePath[:-35],"\n")

    
def pdr():
    pdrlist = []
    pdrStudiesPath = folderPath +'/Pdr/'
    pdrcsv_files = glob.glob(os.path.join(pdrStudiesPath, "*.txt"))

    for f1 in pdrcsv_files:
        data1 = pd.read_csv(f1)    
        data1.rename(columns = {data1.columns[0]:'TEST'}, inplace = True)
        A = data1['TEST'].str.split()
        df2 = pd.DataFrame([A][0])
#         print(df2)
        B = [item for item in A]
        df2 = pd.DataFrame(B, columns = ['PDR Domain Name', 'PDR Record Count','Active Record Count'])
        df2 = df2.astype({"PDR Record Count": int})
        df2 = df2.astype({"Active Record Count": int})
        pdrlist.append(df2)

    with pd.ExcelWriter(filePath,engine="openpyxl",if_sheet_exists="overlay", mode="a",) as writer:
        count=0
        for f in pdrcsv_files:
            sheetname = f.split("/")[-1].split("_")[-3]
            pdrlist[count].to_excel(writer, sheet_name=sheetname,startrow=1 , header=True, startcol=4,index=False)
            count = count+1 
            print("\tData Copied for Pdr Studies: ", sheetname)
    print("\nSuccess...")
    print("Final Excel Sheet generated location : ",filePath[:-35])


filePath = browseFiles() + '/All studies counts_'+time.strftime("%d-%b-%Y.xlsx")
clinstorecsv()
pdr()
color()

	Data Copied for Clinstore Studies:  ACT16105
	Data Copied for Clinstore Studies:  ACT16845
	Data Copied for Clinstore Studies:  ACT16849
	Data Copied for Clinstore Studies:  ACT16902
	Data Copied for Clinstore Studies:  ACT16903
	Data Copied for Clinstore Studies:  EFC15858
	Data Copied for Clinstore Studies:  EFC15935
	Data Copied for Clinstore Studies:  EFC16461
	Data Copied for Clinstore Studies:  EFC16720
	Data Copied for Clinstore Studies:  EFC16723
	Data Copied for Clinstore Studies:  PDY16879
	Data Copied for Clinstore Studies:  TCD14678
	Data Copied for Clinstore Studies:  TCD16210
	Data Copied for Clinstore Studies:  VAT00002B
	Data Copied for Clinstore Studies:  VAT00002
	Data Copied for Clinstore Studies:  VAT00008

Success...
Final Excel Sheet generated location :  C:/Users/894226/OneDrive - Cognizant/Desktop/SaiKorsas/ 

                           TEST
0            [NUL$LIVE, 17, 14]
1           [CG$LIVE, 371, 369]
2       [VS$LIVE, 31634, 22734]
3       [IVRS$LIVE, 6256,